In [13]:
import json
import requests
import pandas as pd

# Because data is stored in json, could not use a simple html beautifulsoup scrape on the website
# https://www.pjm.com/planning/services-requests/interconnection-queues.aspx
# See my stackoverflow question for insight on how to parse data
# https://stackoverflow.com/questions/57315540/missing-html-in-beautifulsoup-scrape/57315740?noredirect=1#comment101205996_57315740

# url was determined by inspecting the page and reviewing the network tab of the developer tools
# searching in the network tab as you navigate the page, you will likely find an api url as shown below 
# under preview, you should see the data you would like returned
url = 'https://services.pjm.com/PJMPlanningApi/api//Queue/GetFilteredQueues?'

# the request payload then contains filters and other information for your data pull
# these payload items can be determined by looking at the request payload under headers
# There are no main filters, but we can add queuefilters to select only the Generation Interconnection projects
# Adding the filters and seeing the request payload change in the network helps to narrow down how to adjust the payload
payload = {'filters': [],
    'rowCount':0,
    'startRow':1,
    "queueFilters": {
    "projectTypes": [
      {
        "id": 0,
        "label": "Generation Interconnection",
        "selected": "true"
      },
      {
        "id": 0,
        "label": "Long-Term Firm Transmission",
        "selected": "false"
      },
      {
        "id": 0,
        "label": "Merchant Transmission",
        "selected": "false"
      },
      {
        "id": 0,
        "label": "Upgrade Request",
        "selected": "false"
      }
    ]
}
          }

# Finally, we need to add some headers to pull data
# We have the origin and api which are listed under the request headers
headers = {
    'Origin': 'https://www.pjm.com',
    'api-subscription-key': 'E29477D0-70E0-4825-89B0-43F460BF9AB4'
}

# to actually get the data, we use a requests.post which will take all of the data in from the page without a limit
json_data = requests.post(url, headers=headers, json=payload).json()

In [15]:
# json_data returns a dictionary. We only want the first entry in the dictionary which contains the queue information
# This is selected by choosing json_data['items']
# We then want to convert this first entry, which is a list of dictionaries, into a data frame.
# The list of dictionaries contains an entry for each queue position and the dictionary represents each variable
PJM_df = pd.DataFrame(json_data['items'])

# Once we have our data stored in a dataframe, we should manipulate it for only the attributes we need
PJM_df=PJM_df[['queueNumber','transmissionOwner','fuelType', 'mw', 'projectStatus', 'projectSubmittedDate',
               'projectedInServiceDate', 'actualInServiceDate', 'projectName', 'stateProvinceName', 'isaStatus',
               'feasibilityStudyStatus', 'systemImpactStudyStatus', 'facilityStudyStatus', 'revisedInServiceDate']]

# Save final DF to csv file for use in ETL process

savepath=r'C:\Users\e988065\Python\Python Files\ETL Process Generator Database\PJM Data Scrape\PJM_Queue.txt'

PJM_df.to_csv(savepath, index=False)